# Patch Tuesday - Vulnerability Prioritization

In [1]:
import pandas as pd

### Vulnerability Prioritization with Weighted Vulnerability Score (WVS)

In this section, we will process the Patch Tuesday vulnerability data to calculate a Weighted Vulnerability Score (WVS) using various factors such as CVSS scores, EPSS scores, CISA KEV status, and more. The steps include normalizing specific fields, defining weights, calculating the WVS, and sorting the vulnerabilities based on their scores.

1. **Load the Data**: Read the processed MSRC EPSS KEV CSV file.
2. **Normalize Fields**: Use mappings to normalize the 'Publicly Disclosed', 'Exploited', and 'Latest Software Release' fields.
3. **Define Weights**: Assign weights to each factor contributing to the overall vulnerability score.
4. **Calculate WVS**: Compute the Weighted Vulnerability Score (WVS) by combining the normalized fields with their respective weights.
5. **Sort and Save**: Sort the vulnerabilities based on the WVS and save the prioritized list to a new CSV file.

#### Weighted Vulnerability Score

The weights assigned to each factor reflect the relative importance of that factor in assessing the overall risk of a vulnerability. These weights are determined based on expert judgment, industry standards, and empirical evidence regarding which factors are most indicative of potential exploitation and impact. The weights used in this example are as follows:

- **Max CVSS (0.30)**: The CVSS (Common Vulnerability Scoring System) score provides a standardized measure of the severity of a vulnerability. It is widely recognized and used in the industry, which justifies its high weight.
- **EPSS (0.30)**: The Exploit Prediction Scoring System (EPSS) estimates the likelihood that a vulnerability will be exploited. Given its predictive value, it is weighted equally to the CVSS score.
- **CISA KEV (0.20)**: The Cybersecurity and Infrastructure Security Agency (CISA) Known Exploited Vulnerabilities (KEV) catalog identifies vulnerabilities that are known to be actively exploited. This factor is crucial but slightly less weighted than CVSS and EPSS.
- **Normalized_Publicly_Disclosed (0.10)**: Publicly disclosed vulnerabilities are known to potential attackers and defenders. Public disclosure increases the urgency for remediation, hence the weight.
- **Normalized_Exploited (0.10)**: If a vulnerability is known to be exploited, it poses a significant risk, which justifies its inclusion in the weighted scoring model.
- **Normalized_Latest_Software_Release (0.05)**: The likelihood of exploitation based on the latest software release provides additional context for prioritization but is considered less critical than other factors, hence its lower weight.

Using a weighted scoring system helps prioritize vulnerabilities in a way that considers both their technical severity and the real-world context of their exploitation, leading to more informed and effective vulnerability management decisions.

In [2]:
# Opt-in to the future behavior
pd.set_option("future.no_silent_downcasting", True)

# Load the CSV file
msrc_epss_kev = pd.read_csv("../../data/patch_tuesday/processed/msrc_epss_kev.csv")


# Function to parse exploit status
def parse_exploit_status(status):
    components = status.split(";")
    parsed_status = {}
    for component in components:
        key, value = component.split(":")
        parsed_status[key.strip()] = value.strip()
    return parsed_status


# Mapping for normalization
publicly_disclosed_mapping = {"Yes": 1, "No": 0}
exploited_mapping = {"Yes": 1, "No": 0}
latest_software_release_mapping = {
    "Exploitation Detected": 1,
    "Exploitation More Likely": 0.75,
    "Exploitation Less Likely": 0.5,
    "Exploitation Unlikely": 0.25,
}

# Normalize the parsed export status values using replace
msrc_epss_kev["Normalized_Publicly_Disclosed"] = (
    msrc_epss_kev["Publicly_Disclosed"]
    .replace(publicly_disclosed_mapping)
    .astype(float)
)
msrc_epss_kev["Normalized_Exploited"] = (
    msrc_epss_kev["Exploited"].replace(exploited_mapping).astype(float)
)
msrc_epss_kev["Normalized_Latest_Software_Release"] = (
    msrc_epss_kev["Latest_Software_Release"]
    .replace(latest_software_release_mapping)
    .astype(float)
)

# Define weights
weights = {
    "CVSS": 0.30,
    "EPSS": 0.30,
    "CISA_KEV": 0.20,
    "Normalized_Publicly_Disclosed": 0.10,
    "Normalized_Exploited": 0.10,
    "Normalized_Latest_Software_Release": 0.05,
}

# Calculate Weighted Vulnerability Score (WVS) with inlined normalization
msrc_epss_kev["Weighted_Vulnerability_Score"] = (
        (msrc_epss_kev["CVSS"] / 10) * weights["CVSS"]
        + msrc_epss_kev["EPSS"] * weights["EPSS"]
        + msrc_epss_kev["CISA_KEV"].astype(float) * weights["CISA_KEV"]
        + msrc_epss_kev["Normalized_Publicly_Disclosed"]
        * weights["Normalized_Publicly_Disclosed"]
        + msrc_epss_kev["Normalized_Exploited"] * weights["Normalized_Exploited"]
        + msrc_epss_kev["Normalized_Latest_Software_Release"]
        * weights["Normalized_Latest_Software_Release"]
)

# Sort by total score
prioritized_df = msrc_epss_kev.sort_values(
    by="Weighted_Vulnerability_Score", ascending=False, na_position="last"
)

# Save to CSV
prioritized_df.to_csv("../../data/patch_tuesday/processed/patch_tuesday_prioritized.csv", index=False)